# Section 1: Stock Selection

## This file defines the stocks that a user can select to feed into the program. Stock selection is driven from a dropdown menu that passes a list as an output. This list will be fed into the IEX Finance and Reuters News API queries. 

In [ ]:
# importing libraries
from pathlib import Path
import pandas as pd
import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
from datetime import datetime, timedelta
from iexfinance.stocks import get_historical_data
import panel as pn
%matplotlib inline
pn.extension("plotly")

In [ ]:
# importing list of companies and converting to DataFrame
sp500_csv = Path("Data/sp500_constituents.csv")
sp500_df = pd.read_csv(sp500_csv)
sp500_df['Company'] = sp500_df['Name']
sp500_df.drop(columns='Sector',inplace=True)
sp500_df.set_index('Company', inplace=True)
sp500_df.head()

In [ ]:
# converting DataFrame to a dictionary of lists
stock_dict = sp500_df.T.to_dict('list')

In [ ]:
# creating an interactive widget that allows the user to select a company
# TODO - export this widget to a panel along with the output widget to create the app's UI
selector_widget = widgets.Dropdown(
    options=stock_dict,
    continuous_update=True)

# creating panel row from widget for export to dashboard
#row = pn.Row(selector_widget)

display(selector_widget)

In [ ]:
# saving the output of the selector value as a variable
stock_selection = selector_widget.value

keyword = f'{stock_selection[0]} AND {stock_selection[1]}'

# Section 2: Stock Data Retrieval

## This section pulls the last month of closing prices of a stock, calculates the daily returns, and exports the daily returns to a DataFrame. This DataFrame will be used as the target values for the machine learning model.

## NOTE: to run this section, you must execute an IEX Finance API key.

In [ ]:
# setting the ticker to the output from the selector
ticker = stock_selection[0]

# setting start and end date for the past four weeks
# 29 days needed instead of 28 days so that we get 28 days of return when we calculate
end_date = datetime.now()
start_date = end_date + timedelta(-31)

# getting data from the API and adding to DataFrame
df = get_historical_data(ticker,start_date,end_date,output_format='pandas')
df.drop(columns=['open','high','low','volume'],inplace=True)
df.head()

In [ ]:
# checking need to clean data
df.isnull().sum()

In [ ]:
# calculating daily returns
returns = df.pct_change() * 100
returns.head(30)

In [ ]:
# checking need to clean data
returns.isnull().sum()

In [ ]:
# cleaning returns data
returns.dropna(inplace=True)
returns.isnull().sum()
returns.rename(columns={'close':'return'},inplace=True)
returns.head(30)

In [ ]:
# exporting data to CSV
returns.to_csv('sample_returns_data.csv')

# Section n: Buy/Sell Recommendations

## This section is used to create the conditional statements that will display the outputs of the ML model, and offer buy/sell recommendations based on them. The outputs and recommendation will be displayed in a widget that will be exported to a panel dashboard along with the input widget to form the user interface.

In [ ]:
# importing test data
# TODO - this will not be included in final code - this will be replaced with output from ML model below
test_csv = Path("test_model_output.csv")
test_df = pd.read_csv(test_csv)

test_df.head(8)

In [ ]:
# creating accuracy selector widget for setting model accuracy threshold to feed into conditional statements
accuracy_selector = widgets.IntSlider(
    value=75,
    min=0,
    max=100,
    step=1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

In [ ]:
# saving the output of the accuracy value as a variable
accuracy_value = accuracy_selector.value

In [ ]:
# referencing a row in the DataFrame
# TODO - link up to actual outputs from ML model
company_selector = 7

# variables for recommendation bounds
strong_buy_lower = .1
buy_upper = strong_buy_lower
buy_lower = .05
pos_hold_upper = buy_lower
pos_hold_lower = 0
neg_hold_upper = pos_hold_lower
neg_hold_lower = -.05
sell_upper = neg_hold_lower
sell_lower = -.1

# creating conditional statement to determine buy/sell recommendations
# TODO - link to interactive widgets that allow users to set their bands

if (test_df.loc[company_selector][5]*100) > accuracy_value:
    
    if test_df.loc[company_selector][3] >= strong_buy_lower:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price increase from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: STRONG BUY.'
    elif buy_lower <= test_df.loc[company_selector][3] < buy_upper:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price increase from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: BUY.'
    elif pos_hold_lower < test_df.loc[company_selector][3] < pos_hold_upper:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price increase from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: HOLD.'
    elif test_df.loc[company_selector][3] == 0:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be no price change. Our recommendation: HOLD.'
    elif neg_hold_lower < test_df.loc[company_selector][3] < neg_hold_upper:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price decrease from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: HOLD.'
    elif sell_lower < test_df.loc[company_selector][3] <= sell_upper:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price decrease from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: SELL.'
    else:
        output = f'With a composite news sentiment score of {test_df.loc[company_selector][4]}, there is a {test_df.loc[company_selector][5]*100}% chance there will be a {test_df.loc[company_selector][3]*100}% price decrease from ${test_df.loc[company_selector][1]} to ${test_df.loc[company_selector][2]}. Our recommendation: STRONG SELL.'
        
else: 
    output = f'Model accuracy is only {test_df.loc[company_selector][5]*100}%. We do not have enough confidence to provide an investment recommendation given the uncertainty.'
    
output

# Section n: User Interface

## This section is used to create the ipywidgets that will be used to trigger running the model based on user inputs, and display the model outputs + recommendations. 


In [ ]:
# creating input widget
layout = Layout(border='solid 1.5px')

selector_title = widgets.Output(layout=layout)
selector_title.append_stdout('Choose Company:')

accuracy_title = widgets.Output(layout=layout)
accuracy_title.append_stdout('Required Model Accuracy (%):')

run_button = widgets.Button(description="Run Model",layout=layout)

# TODO - update this function so that the whole model runs again with the new inputs when the user clicks the button
def on_button_clicked(b):
    display(output_widget)
    
run_button.on_click(on_button_clicked)

input_widget = widgets.VBox([selector_title, selector_widget, 
                              accuracy_title, accuracy_selector, 
                              run_button],
                             layout=layout
                            )

In [ ]:
# creating output widget

output_text = widgets.Output()
output_text.append_stdout(output)

recommendation_title = widgets.Output(layout=layout)
recommendation_title.append_stdout('Your Recommendation:')

output_widget = widgets.VBox([recommendation_title, output_text])                            

In [ ]:
# display widgets
display(input_widget)